In [142]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [143]:
#Definimos la ruta en donde queremos guardar los archivos
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
ruta_pagos = '/home/estadistico/Documents/Erick'
mes = 20201201
mes2 = 'Diciembre2020'
mes3 = pd.to_datetime('01/12/2020',dayfirst=True)
prim_sem_mes = 49

In [144]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contrasena = 'estadisticas8474'
base = 'procesos_externos'

In [145]:
#Lectura pagos Bradesco c0
pagos_brad_c0 = pd.read_excel(''+ruta_pagos+'/Bradesco/Facturacion_c0/Facturacion c0.xlsx',converters={'TARJETA':str})
pagos_brad_c0 = pagos_brad_c0.drop_duplicates()
#pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
#pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
#pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [146]:
#Lectura pagos Bradesco extrajudicial
pagos_brad_extra = pd.read_excel(''+ruta_pagos+'/Bradesco/Facturacion_extra/Facturacion extra.xlsx',converters={'TARJETA':str})
pagos_brad_extra = pagos_brad_extra.drop_duplicates()
#pagos_brad2 = pd.pivot_table(pagos_brad,index=['NUMERO_TARJETA','FECHA_PAGO','TIPO_ASIGANCION'],values=['MONTO'],aggfunc=np.sum)
#pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())
#pagos_brad2 = pagos_brad2.loc[pagos_brad2['FECHA_PAGO']>=mes3]

In [147]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [148]:
#Hacemos la conexion con el servidor\n",
try: 
    conn = mysql.connector.connect(user=usuario,
                               password=contrasena,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

In [149]:
#Funcion para hacer la consulta sql, ya sea de gestion o de asignacion en donde requerimos tres parametros, la tabla de donde vamos a sacar la consulta, en caso de asignacin requerimos los otros dos parametros fecha inicial y fecha final
def consulta_sql(table,from_date=None,to_date=None,campana=None):
    if campana is None:
        if from_date is None:
            sql_mes = cursor.callproc(table,)
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
        else:
            sql_mes = cursor.callproc(table, [from_date,to_date])
            for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    else:
        sql_mes = cursor.callproc(table, [from_date,to_date,campana])
        for result in cursor.stored_results():
                sql_mes = pd.DataFrame(result.fetchall())
    sql_mes = sql_mes.drop_duplicates()
    return sql_mes

# Bradesco

In [150]:
#Inicio y fin de meses que nos interesa conocer
inicio_mes = [20200101,20200201,20200301,20200401,20200501,20200601,20200701,20200801,20200901,20201001,20201101]
fin_mes = [20200131,20200228,20200331,20200430,20200531,20200630,20200731,20200831,20200930,20201031,20201130]

In [151]:
#Lectura de cada uno de los meses
gestion_brad = pd.DataFrame()
for i,j in zip(inicio_mes,fin_mes):
    gestion_brad_mes = consulta_sql('bradescard_rpt_gestiones_detallado',i,j)
    gestion_brad = pd.concat([gestion_brad,gestion_brad_mes],ignore_index=True)
    print(i)

20200101
20200201
20200301
20200401
20200501
20200601
20200701
20200801
20200901
20201001
20201101


In [152]:
#Asignacion de nombre de columnas
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
promesas_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']

In [153]:
#Ajustamos parametros para la gestion
promesas_brad['monto_promesa'] = pd.to_numeric(promesas_brad['monto_promesa'])
promesas_brad = promesas_brad.drop_duplicates(subset=['credito','usuario','fecha_promesa','monto_promesa'])
promesas_brad['anio'] = [x[:4] for x in promesas_brad['fecha_promesa']]
promesas_brad = promesas_brad.loc[(promesas_brad['anio']=='2020')]
promesas_brad['fecha_promesa'] = pd.to_datetime(promesas_brad['fecha_promesa'],dayfirst=False)
promesas_brad['mes'] = pd.DatetimeIndex(promesas_brad['fecha_promesa']).month
promesas_brad['credito_unico'] = promesas_brad['credito'].map(str)+'-'+promesas_brad['mes'].map(str)+'-'+promesas_brad['anio'].map(str)
promesas_brad.head(1)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,dictamen,accion,resultado,accion_resultado,fecha_promesa,monto_promesa,comentarios,anio,mes,credito_unico
20,6604600,1,4,5063290226839017,ORONA PONCE ANGEL ALFREDO,6674234755,celular,2020-01-31 20:43:22,YHERNANDEZBA,ALEJANDRA YESENIA HERNANDEZ BITE,PROMESA,LLAMADA A TITULAR,PROMESA DE PAGO LIQUIDACION,LLAMADA A TITULAR-PROMESA DE PAGO LIQUIDACION,2020-01-31,2250.0,6674234755 se agenda pago por:2250 al dia 318...,2020,1,5063290226839017-1-2020


In [154]:
#Concatenamos los pagos de brad
pagos_brad = pd.concat([pagos_brad_c0,pagos_brad_extra],ignore_index=True)
pagos_brad = pagos_brad.loc[pagos_brad['FECHA'].map(str)!='00:00:00']
pagos_brad['YEAR'] = pagos_brad['FECHA'].dt.year
pagos_brad['MONTH'] = pagos_brad['FECHA'].dt.month
pagos_brad['credito_unico'] = pagos_brad['TARJETA'].map(str)+'-'+pagos_brad['MONTH'].map(str)+'-'+pagos_brad['YEAR'].map(str)
pagos_brad.head(1)

,PRODUCTO,TARJETA,PAGO,FECHA,PORCENTAJE,COMISION,BUCKET,DESPACHO,MES,YEAR,MONTH,credito_unico
0,PLCC,0002869002607825000,1542.0,2020-07-30 00:00:00,0.1,154.2,5,ENRIQUEZ,2020-07-01,2020,7,0002869002607825000-7-2020


In [155]:
pagos_brad2 = pd.pivot_table(pagos_brad,index=['TARJETA','FECHA','BUCKET','credito_unico','MES'],values=['PAGO','COMISION'],aggfunc=np.sum)
pagos_brad2 = pd.DataFrame(pagos_brad2.to_records())

In [161]:
#Hacemos la union entre promesas y pagos
promesas_pagos_brad = pd.merge(promesas_brad,pagos_brad2,how='left',on='credito_unico')
promesas_pagos_brad = promesas_pagos_brad.drop_duplicates(subset=['credito','fecha_promesa','FECHA','PAGO','usuario'])
promesas_pagos_brad['usuario'] = [x.rstrip() for x in promesas_pagos_brad['usuario']]
promesas_pagos_brad.head(1)

,folio_gestion,firma_id,unegocio_id,credito,nombre_credito,telefono,tipo_telefono,fecha_gestion,usuario,nombre_usuario,...,comentarios,anio,mes,credito_unico,TARJETA,FECHA,BUCKET,MES,COMISION,PAGO
0,6604600,1,4,5063290226839017,ORONA PONCE ANGEL ALFREDO,6674234755,celular,2020-01-31 20:43:22,YHERNANDEZBA,ALEJANDRA YESENIA HERNANDEZ BITE,...,6674234755 se agenda pago por:2250 al dia 318...,2020,1,5063290226839017-1-2020,5063290226839017,2020-01-31,A,2020-01-01,607.5,2250.0


In [165]:
gestor_mes = pd.pivot_table(promesas_pagos_brad,index=['usuario'],values=['monto_promesa'],columns=['mes'],aggfunc=np.sum)
gestor_mes = pd.DataFrame(gestor_mes.to_records())
gestor_mes = gestor_mes[gestor_mes.isnull().sum(axis=1) < 8]

In [163]:
def cap_data(df):
    for col in df.columns:
        print("capping the ",col)
        if (((df[col].dtype)=='float64') | ((df[col].dtype)=='int64')):
            percentiles = df[col].quantile([0.01,0.99]).values
            df[col][df[col] <= percentiles[0]] = percentiles[0]
            df[col][df[col] >= percentiles[1]] = percentiles[1]
        else:
            df[col]=df[col]
    return df

In [166]:
final_df=cap_data(gestor_mes)

capping the  usuario
capping the  ('monto_promesa', 1)
capping the  ('monto_promesa', 2)
capping the  ('monto_promesa', 3)
capping the  ('monto_promesa', 4)
capping the  ('monto_promesa', 5)
capping the  ('monto_promesa', 6)
capping the  ('monto_promesa', 7)
capping the  ('monto_promesa', 8)
capping the  ('monto_promesa', 9)
capping the  ('monto_promesa', 10)
capping the  ('monto_promesa', 11)
capping the  ('monto_promesa', 12)
capping the  ('monto_promesa', 'All')
